In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Loading Data

In [4]:
BASE_DIR = Path("/kaggle/input/playground-series-s3e2/")

train = pd.read_csv(BASE_DIR / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_DIR / "test.csv").drop(columns="id")

# Preprocessing

In [5]:
df = pd.concat([train.drop(columns=["stroke"]), test], axis=0).reset_index(drop=True)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked


# Feature Engineering

In [6]:
def diabetes_indicator(avg_glucose_level):
    if avg_glucose_level <= 99:
        return "normal"
    elif avg_glucose_level >= 100 and avg_glucose_level <= 125:
        return "prediabetic"
    elif avg_glucose_level <= 200:
        return "type1"
    else:
        return "type2"

df.avg_glucose_level.map(diabetes_indicator).value_counts()

normal         20098
prediabetic     4002
type1            916
type2            492
Name: avg_glucose_level, dtype: int64

In [7]:
df["diabetes"] = df.avg_glucose_level.map(diabetes_indicator)
df.head(3)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,diabetes
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,normal
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,normal
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,prediabetic


# One hot encoding

In [8]:
df = pd.get_dummies(df)
df.head(3)

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,...,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,diabetes_normal,diabetes_prediabetic,diabetes_type1,diabetes_type2
0,28.0,0,0,79.53,31.1,0,1,0,0,1,...,0,1,0,0,1,0,1,0,0,0
1,33.0,0,0,78.44,23.9,0,1,0,0,1,...,1,0,0,1,0,0,1,0,0,0
2,42.0,0,0,103.00,40.3,1,0,0,0,1,...,1,0,1,0,0,0,0,1,0,0


# Preprocessing v2

In [11]:
X = df.iloc[:-len(test), :]
test_new = df.iloc[-len(test):, :]
y = train.stroke

In [9]:
xgb_params = {'n_estimators': 150,
                 'max_depth': 4,
                 'learning_rate': 0.10513333456836436,
                 'min_child_weight': 8,
                 'gamma': 0.002126415095687836,
                 'subsample': 0.5470092690254375,
                 'colsample_bytree': 0.6296509957360737,
                 'reg_alpha': 0.0003558325520757654,
                 'reg_lambda': 0.6330475930599759}

In [10]:
xgb_model = xgb.XGBClassifier(**xgb_params)

In [12]:
xgb_model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.6296509957360737, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              gamma=0.002126415095687836, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.10513333456836436, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=4, max_leaves=0,
              min_child_weight=8, missing=nan, monotone_constraints='()',
              n_estimators=150, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0.0003558325520757654,
              reg_lambda=0.6330475930599759, ...)

In [15]:
y_pred = xgb_model.predict_proba(test_new)[:, 1]
y_pred

array([0.03234255, 0.18512149, 0.00081496, ..., 0.00072294, 0.00219616,
       0.00048123], dtype=float32)

In [16]:
# since we dropped id column from our test dataframe earlier but we need it for submission file so we're reloading it
test_idx = pd.read_csv(BASE_DIR / "test.csv").id
test_idx

0        15304
1        15305
2        15306
3        15307
4        15308
         ...  
10199    25503
10200    25504
10201    25505
10202    25506
10203    25507
Name: id, Length: 10204, dtype: int64

In [17]:
submission = pd.DataFrame(data={"id": test_idx, "stroke": y_pred})
submission.head()

,id,stroke
0,15304,0.032343
1,15305,0.185121
2,15306,0.000815
3,15307,0.084764
4,15308,0.003461


In [18]:
submission.to_csv("submission.csv", index=False)